In [1]:
import matplotlib.pyplot as plt # visualization
import missingno as msno # missing data visualization
import numpy as np # array manipulation
import pandas as pd # dataframe manipulation
import scipy # statistical tests
import seaborn as sns # visualization
from zipfile import ZipFile # reading zip file
import random

In [ ]:
train_zip_file_path = "D:\\Winton-Challenge\\train.csv.zip"

train_file = ZipFile(train_zip_file_path)
train = pd.read_csv(train_file.open("train.csv"))

In [ ]:
train.describe()

In [ ]:
#To make inferences about Feature
train.iloc[:, 1:26].hist(bins=50, figsize=(30,20))

plt.show()

# Features
1. 1-25 are values that govern stock points of a particular stock
2. Some are categorical some are discrete
3. There are many missing values for some features -> WE could try to impute this either by using **k-nearest neighbour(expensive)** or **other clusterisng methods(mean, median or mode)** or **Scikit-learn's IterativeImputer**

> **Hypothesis** : Feautre_1, Feature_5, Feature_10, Feature_13, Feature_16 and Feature_20 are categorical

In [ ]:
#To make inference about stock's nature i.e. Returns
def plot_random_stock(train, feature_slice=(28,207)):
    rng = np.random.default_rng()
    rando_id = rng.integers(0, len(train))
    stock_val_intraday = train.iloc[rando_id, slice(*feature_slice)].to_numpy()
    x = np.arange(stock_val_intraday.size)
    
    plt.figure(figsize=(10, 5))
    plt.plot(x, stock_val_intraday, '-o', markersize=4, linewidth=1.5)
    plt.title(f"Intraday Returns of Stock #{rando_id}")
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.grid(alpha=0.3)
    plt.show()

plot_random_stock(train)

# Returns
1. Ret_MinusTwo:  this is the return from the close of trading on day D-2 to the close of trading on day D-1 (i.e. 1 day)
2. Ret_MinusOne: this is the return from the close of trading on day D-1 to the point at which the intraday returns start on day D (approximately 1/2 day)
3. Ret_2 to Ret_120: these are returns over approximately one minute on day D. Ret_2 is the return between t=1 and t=2. 
4. Ret_121 to Ret_180: intraday returns over approximately one minute on day D. These are the target variables you need to predict as {id}_{1-60}. 
5. Ret_PlusOne: this is the return from the time Ret_180 is measured on day D to the close of trading on day D+1. (approximately 1 day). **This is a target variable you need to predict as {id}_61.** 
6. Ret_PlusTwo: this is the return from the close of trading on day D+1 to the close of trading on day D+2 (i.e. 1 day) **This is a target variable you need to predict as {id}_62.** 

> Different Stocks have different type of returns( variational, oscillating between two-three points, not varying at all)

> This could indicate category of different stocks that follow a similiar rule

# Weights 
> This is used for calculating **Mean Absolute Error** for the predictions
1. Weight_Intraday: weight used to evaluate intraday return predictions Ret 121 to 180
2. Weight_Daily: weight used to evaluate daily return predictions (Ret_PlusOne and Ret_PlusTwo).
